In [1]:
import tensorflow as tf
import numpy as np
import os
from tqdm import tqdm

slim = tf.contrib.slim

가르쳐야 하는 것들에 대해서 정리해보자.

1. Tensorflow Model을 어떻게 저장하고 불러올 것인가? 
   * 이걸 똑바로 이해하기 위해서는 Tensorflow가 어떤 식으로 이루어져 있는지 알아야 함.
     왜 checkpoint와 meta_graph로 나뉘어져 있는지 이해하고, 각각 어떤 정보를 담고 있는지를 알아야 함


# Tensorflow의 설계 형태

우선 transfer learning을 하기 전에, 먼저 텐서플로우의 구조부터 이해해보도록 하자. 


텐서플로우는 빌딩구조와 실행구조가 분리되어 있다.
* 빌딩구조 : graph = tensors(edges) + operations(nodes)
* 실행구조 : session


## 1. 빌딩구조

graph는 nodes와 edges로 구성된 하나의 프로그램 모듈.

* **nodes** : the units of computation
* **edges** : the units of data that flow between operations


![](../src/images/tensorflow 구조.png)

tensorflow의 graph는 node들의 집합이다. node들은 아래와 같은 요소들로 정의되어 있다.

| 구성요소  | 간단 설명  | description | 
|---   |          ---|---|
| name  | 고유한 지정 이름| unique identifier that's not used by any other nodes in the graph |
| op    | 연산 종류(conv, lstm, add 등) | what operators to run (for example : `Add`, `MatMul`, `Conv2D`) |
| input | 연산을 적용할 node의 name     | a list of strings, each one of which is the name of another node  |
| device | 연산할 device의 종류  | defines where to run a node in a distributed environment, or when you want to force the operation onto CPU or GPU. |
| attr | 연산의 속성값(conv filter 갯수 등) | {key : value} holding all the attributes of a node. the permanent properties of nodes, things that don't change at runtime |


중요한 것 중 하나는 weights가 여기에 포함되어 있지 않다는 사실이다. 학습된 weights들은 어디에 포함되어 있을까? 

-> 바로 session(checkpoints)


## 2. 실행구조

* fetch : 연산의 결과를 가져오는(fetch) 방법
* feed :  placeholder에 값을 넣어 실행하는 방법

``
tf.Session.run(fetches, feed_dict=None, options=None, run_metadata=None)
``




세션 run의 구조
![](../src/images/sess_run.png)

실제로 텐서플로우에서 값들을 저장되고 처리되는 단계는 실행구조(Session)에서 모두 이루어진다. 

graph는 이를 추상화시켜 놓은 것에 불과하다. 

그렇기 때문에 우리가 작업을 할 때는 변수를 초기화해주는 작업이 꼭 필요하다. 

In [2]:
tf.reset_default_graph()

x = tf.Variable(20, name='x')
y = tf.Variable(10, name='y')
z = tf.add(x,y)


with tf.Session() as sess:
    # 변수 초기화
    # 변수를 초기화하지 않으면, 
    # FailedPreconditionError 발생
    x.initializer.run() # x를 초기화
    y.initializer.run() # y를 초기화

    print(sess.run(z))

30


역으로 우리가 기존에 저장된 모델을 불러온다는 것은 어떤 식으로 이루어져야 할까?

우선 우리는 불러올 모델에 대한 graph 구조가 필요하고, 

학습된 weights들의 값을 해당 graph로 할당해주어야 한다. 

학습된 weights들의 값을 해당 graph로 할당해주어야 하기 때문에, 

우리는 session에다가 주입을 해야 한다.

### 간단한 모델을 만들어서 학습을 시켜보자

In [3]:
from tensorflow.examples.tutorials.mnist import input_data

In [4]:
# Load mnist dataset
mnist = input_data.read_data_sets('./')

n_inputs = mnist.train.images.shape[1]
n_outputs = 10

learning_rate = 0.001

n_epochs = 20
batch_size = 64

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./train-labels-idx1-ubyte.gz
Extracting ./t10k-images-idx3-ubyte.gz
Extracting ./t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [5]:
tf.reset_default_graph()

graph = tf.get_default_graph() # 현재 default로 설정된 Tensorflow의 graph로 가져옴

# input placeholder
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None,), name='y')

leaky_relu = lambda x: tf.nn.leaky_relu(x) # Activation Function
xavier_init = tf.initializers.variance_scaling() # Initialization for last fully-connected layer
he_init = tf.initializers.variance_scaling(scale=2.) # Initialization for all layer except last layer

with tf.variable_scope('cnn'):
    # 28,28,1로 변경하여, convolution 연산이 가능하도록 함
    _input = tf.reshape(X, (-1,28,28,1)) 

    with tf.variable_scope('conv_block1'):
        conv_layer = tf.layers.conv2d(_input, 8, (3,3), 
                                        strides=(1,1),padding='same',
                                        activation=leaky_relu, name='conv',
                                        kernel_initializer=he_init)
        max_pool = tf.layers.max_pooling2d(conv_layer,(2,2),(2,2),
                                             name='pool', padding='same')

    with tf.variable_scope('conv_block2'):
        conv_layer = tf.layers.conv2d(max_pool, 8, (3,3), 
                                        strides=(1,1),padding='same',
                                        activation=leaky_relu, name='conv',
                                        kernel_initializer=he_init)
        max_pool = tf.layers.max_pooling2d(conv_layer,(2,2),(2,2),
                                             name='pool', padding='same')

    with tf.variable_scope('conv_block3'):
        conv_layer = tf.layers.conv2d(max_pool, 8, (3,3), 
                                        strides=(1,1),padding='same',
                                        activation=leaky_relu, name='conv',
                                        kernel_initializer=he_init)
        max_pool = tf.layers.max_pooling2d(conv_layer,(2,2),(2,2),
                                             name='pool', padding='same')

    with tf.variable_scope('conv_block4'):
        conv_layer = tf.layers.conv2d(max_pool, 8, (3,3), 
                                        strides=(1,1),padding='same',
                                        activation=leaky_relu, name='conv',
                                        kernel_initializer=he_init)
        max_pool = tf.layers.max_pooling2d(conv_layer,(2,2),(2,2),
                                             name='pool', padding='same')

    with tf.variable_scope('conv_block5'):
        conv_layer = tf.layers.conv2d(max_pool, 8, (3,3), 
                                        strides=(1,1),padding='same',
                                        activation=leaky_relu, name='conv',
                                        kernel_initializer=he_init)
        max_pool = tf.layers.max_pooling2d(conv_layer,(2,2),(2,2),
                                             name='pool', padding='same')

    with tf.variable_scope("fully_connected"):
        avg_pool = tf.reduce_mean(max_pool, axis=[1,2],name="global_average_pooling")
        flatten = tf.layers.flatten(avg_pool,name='flatten')
        dense_layer = tf.layers.dense(flatten, 32,
                                      kernel_initializer=he_init,
                                      activation=leaky_relu, 
                                      name='fc_layer')
        logits = tf.layers.dense(dense_layer, n_outputs, 
                                 kernel_initializer=xavier_init,
                                 name='output')

    with tf.variable_scope("loss"):
        # sparse_softmax_cross_entropy_with_logits
        # 소프트맥스 활성화 함수를 적용한 다음 크로스 엔트로피를 계산
        xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(
            labels=y, logits=logits)
        # labels : 0~n_classes-1 사이의 정수로 된 label
        # logits : 소프트맥스 활성화 함수로 들어가기 전의 네트워크 출력
        loss = tf.reduce_mean(xentropy, name='loss')

    with tf.variable_scope('eval'):
        correct = tf.nn.in_top_k(logits, y, 1)
        accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss)

* GLOBAL_VARIABLES
> Key to collect Variable objects that are global (shared across machines). Default collection for all variables, except local ones.

* LOCAL_VARIABLES
> Key to collect local variables that are local to the machine and are not saved/restored.

In [6]:
sess = tf.Session()
# 보통은 global_variables_initializer만 존재해도 상관없으나
# 간혹 안될 경우가 있어서, tf.local_variables_initializer를 추가
init = [tf.global_variables_initializer(),
        tf.local_variables_initializer()]
sess.run(init)
epoch_bar = tqdm(range(n_epochs))
for epoch in epoch_bar:
    for iteration in range(mnist.train.num_examples // batch_size):
        X_batch, y_batch = mnist.train.next_batch(batch_size)
        sess.run(training_op, feed_dict={X: X_batch,y: y_batch})

    acc_train = sess.run(accuracy,feed_dict={X: X_batch,y: y_batch})
    acc_val = sess.run(accuracy,feed_dict={X: mnist.validation.images,
                                           y: mnist.validation.labels})
    epoch_bar.set_description(
        "{} epoch | Train accuracy: {:2.2f} Validation accuracy: {:2.2f}\n".format(epoch, acc_train, acc_val))

0 epoch | Train accuracy: 0.28 Validation accuracy: 0.21
0 epoch | Train accuracy: 0.28 Validation accuracy: 0.21
1 epoch | Train accuracy: 0.39 Validation accuracy: 0.40
1 epoch | Train accuracy: 0.39 Validation accuracy: 0.40
2 epoch | Train accuracy: 0.52 Validation accuracy: 0.60
2 epoch | Train accuracy: 0.52 Validation accuracy: 0.60
3 epoch | Train accuracy: 0.70 Validation accuracy: 0.74
3 epoch | Train accuracy: 0.70 Validation accuracy: 0.74
4 epoch | Train accuracy: 0.86 Validation accuracy: 0.82
4 epoch | Train accuracy: 0.86 Validation accuracy: 0.82
5 epoch | Train accuracy: 0.84 Validation accuracy: 0.86
5 epoch | Train accuracy: 0.84 Validation accuracy: 0.86
6 epoch | Train accuracy: 0.91 Validation accuracy: 0.88
6 epoch | Train accuracy: 0.91 Validation accuracy: 0.88
7 epoch | Train accuracy: 0.81 Validation accuracy: 0.89
7 epoch | Train accuracy: 0.81 Validation accuracy: 0.89
8 epoch | Train accuracy: 0.91 Validation accuracy: 0.90
8 epoch | Train accuracy: 0.91 

In [7]:
model_variables = {}
with graph.as_default():
    # 학습된 모든 variable들은 train_variables에 담겨져 있음
    # train_variables 순회하면서 가져올 수 있다.
    
    #train_variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,scope="cnn/conv_block[1-5]/conv/kernel")
    train_variables = tf.global_variables()
    for variable in train_variables:
        tensor_name, tensor_index = variable.name.split(":")
        # 저장된 weights 가져오기
        weights = variable.read_value().eval(session=sess)
        model_variables[tensor_name] = weights

c.f)

모든 변수들은 자동적으로 그들이 만들어진 graph에 쌓입니다. 

기본적으로, 생성자는 그래프 컬렉션(graph collection) GraphKeys.VARIABLES에 변수를 추가합니다. 

편의 함수인 all_variables()은 컬렉션의 내용을 반환합니다.


머신 러닝 모델을 만들 때, 학습 가능한 모델 매개변수를 가지고 있는 변수와 global step 변수과 같이 학습 단계를 계산하기 위한 다른 변수로 구분하는 것은 종종 편리합니다. 

이것을 더 쉽게 하기위해, 변수 생성자는 trainable=<bool> 매개변수를 지원합니다. 
    
True일 때 새로운 변수는 그래프 컬렉션 GraphKeys.TRAINABLE_VARIABLES에 추가됩니다. 

편의 함수 trainable_variables()는 이 컬렉션의 내용을 반환합니다. 

다양한 Optimizer 클래스는 이 컬렉션을 최적화(optimize) 변수의 기본 리스트로 사용합니다.

우리가 모델을 저장하고 복원하기 위해서는 두 가지(graph, variables)를 저장하고 복원할 수 있어야 한다.

## 1. Graph Save & Load

그래프는 기본적으로 `tf.train.export_meta_graph`와 `tf.train.import_meta_graph`로 관리된다. Tensorflow는 graph를 저장할 때 기본적으로 `MetaGraph`로 저장된다. graph를 관리하는 데에는 단순히 Graph의 node와 edge 뿐만 아니라 그 외 실행환경에 대한 정보도 같이 있어야 한다고 판단하기 때문이다.

그렇기 때문에 Tensorflow의 버전 정보, 저장한 Saver에 대한 정보, 그외 CollectionDef에 대한 정보등이 있다. MetaGraph가 있으면 나중에는 처음부터 모델을 구축하지 않고 가져와서 계속 훈련도 가능하다

MetaGraph에 있는 것

| 종류 | 설명 |
|----|------|
|GraphDef| 그래프를 묘사 |
|MetaInfoDef| 버전과 기타 사용자 정보 같은 메타 정보를 포함  |
|SaverDef| saver에 대한 정보 |
|CollectionDef| `Variables`, `QueueRunners`와 같은 추가적인 요소를 더 자세히 설명 |




metagraph를 Import할 때는 implicit하게 현재 default graph로 import된다.


참고자료 
* [텐서플로우 한글 문서](https://tensorflowkorea.gitbooks.io/tensorflow-kr/content/g3doc/how_tos/meta_graph/)

In [8]:
# save graph
save_dir = "./saves/"
graph_path = os.path.join(save_dir,"graph.meta")

tf.train.export_meta_graph(graph_path, graph=graph);

In [9]:
# load graph
graph2 = tf.Graph()
print("before import_meta_graph : \n",graph2.get_operations()) # 비어있음

with graph2.as_default():
    meta_graph = tf.train.import_meta_graph(graph_path)
    
print("After import_meta_graph : \n",graph2.get_operations()[:3]) # 차있음

before import_meta_graph : 
 []
After import_meta_graph : 
 [<tf.Operation 'X' type=Placeholder>, <tf.Operation 'y' type=Placeholder>, <tf.Operation 'cnn/Reshape/shape' type=Const>]


## 2. variable Save & Load

기본적으로 Variable에 대한 관리는 Saver가 담당한다. Variable은 학습에 따라 계속 변하는 Time-variant한 값이기 때문에, 기본적으로는 `checkpoints` 개념으로 저장되고 로드된다. 아래는 공식문서에서 가져온 Saver에 대한 개념이다.

class Saver
----

Saves and restores variables.

See Variables for an overview of variables, saving and restoring.

The Saver class adds ops to save and restore variables to and from checkpoints. It also provides convenience methods to run these ops.

Checkpoints are binary files in a proprietary format which map variable names to tensor values. The best way to examine the contents of a checkpoint is to load it using a Saver.


saving할 때 나오는 것들은 아래와 같다.

| 확장자 | 설명 |
|---|---|
| .meta  | containing the graph structure |
| .data  | containing the values of variables |
| .index | identifying the checkpoint |
| .checkpoint | a protocol buffer with a list of recent checkpoints |

핵심은 .meta 파일과 .checkpoint 파일로, 이 두 파일이 있으면 이전 Session으로 복구가 가능하다 .


In [10]:
# Save Variable(Session)
save_dir = "./saves/" # 저장할 디렉토리
os.makedirs(save_dir,exist_ok=True) 
save_path = os.path.join(save_dir,"model")

saver = tf.train.Saver()
init = tf.global_variables_initializer()
sess.run(init)
saver.save(sess, save_path=save_path)

'./saves/model'

In [11]:
# Load Variable(Session)
with tf.Graph().as_default() as restored_graph:
    meta_graph = tf.train.import_meta_graph(graph_path)
    restored_sess = tf.Session(graph=restored_graph)
    meta_graph.restore(restored_sess, save_path=save_path)

INFO:tensorflow:Restoring parameters from ./saves/model
